In [1]:
import numpy as np
from matplotlib import pyplot as pl
import os
import scipy.io.wavfile as wav
from python_speech_features import mfcc
import utils.mlp_backprop_momentum as mlp
import utils.k_fold_cross_validation as cv

%matplotlib inline

In [2]:
PATH = 'vowels/'

MALE = 'm'
FEMALE = 'f'
KID = 'k'


In [3]:
mfccs = []

for filename in os.listdir(PATH):
    if filename.endswith('.wav'):
        sample_rate, X = wav.read(os.path.join(PATH, filename))
        ceps = mfcc(X, samplerate=sample_rate, nfft=1024)
        
        speaker = ''
        if filename[2] == 'm':
            speaker = MALE
        elif filename[2] == 'f':
            speaker = FEMALE
        elif filename[1] == 'k':
            speaker = KID
        else:
            continue
        
        for window in ceps:
            i = 0          
            for sample in window:
                mfccs.append((sample, i, speaker))
                i+=1
# mfccs contient désormais toutes les valeurs de mfcc, ainsi que la position du mfcc dans la window et le type de speaker


In [4]:
# learning 

DATASET_SIZE = len(mfccs)
SPREAD = 0.7
dataset = create_dataset(DATASET_SIZE, SPREAD)
N_INITS = 10
EPOCHS = 200 #nb d'iterations
N_NEURONS = [2, 4, 8, 16, 32]
LEARNING_RATE = 0.001
MOMENTUM = 0.5

MSE_train, MSE_test, conf_mat = cv.k_fold_cross_validation(nn,
                                                          dataset,
                                                          k=K,
                                                          learning_rate=LEARNING_RATE,
                                                          momentum=MOMENTUM,
                                                          epochs=EPOCHS,
                                                          threshold=0.0)

Testing dataset with variance: 0.4


NameError: name 'create_dataset' is not defined